In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
'''

from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [2]:
# Read musical data
music_chord = pd.read_csv('../tmp/Chopin_Etude_Op_10_n_1_grades_chords.csv')

In [3]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = '../tmp'
writer = tf.summary.FileWriter(logs_path)

# training_data = read_data(training_file)


training_data = music_chord['grades']
print("Loaded training data...")

def build_dataset(words):
    # A very sad way of creating a dictionary
    count = np.unique(words)
    dictionary = dict()
    for word in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

Loaded training data...


In [4]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [5]:
# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 10

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):
    
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])
    print(x)

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    print(x)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    print(rnn_cell)
    
    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    print(outputs)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
pred = RNN(x, weights, biases)

Tensor("Reshape:0", shape=(?, 10), dtype=float32)
[<tf.Tensor 'split:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:3' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:4' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:5' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:6' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:7' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:8' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:9' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_2:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_5:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_8:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_11:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_ce

In [7]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        words = input(prompt)
        print(words)
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(100):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = reverse_dictionary[onehot_pred_index]
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
                print(sentence)
        except:
            print("Word not in dictionary")
    tf.saved_model.simple_save(
            session, '../tmp', symbols_in_keys, symbols_out_onehot
        )

Iter= 1000, Average Loss= 7.526896, Average Accuracy= 0.20%
["('X', 'V2', 'V1')", "('V2', 'IV4', 'V1')", "('VII3', 'V2', 'V1')", "('V2', 'II4', 'V1')", "('V2', 'X', 'V1')", "('IV3', 'V2', 'V1')", "('VII2', 'V2', 'V1')", "('V2', 'V1')", "('X', 'V2', 'V1')", "('III3', 'V2', 'V1')"] - [('V2', 'V3', 'V1')] vs [('II5', 'V2', 'V1')]
Iter= 2000, Average Loss= 5.678412, Average Accuracy= 1.60%
["('III1', 'III2', 'III6')", "('III1', 'III2', 'VII5')", "('III1', 'X', 'III2')", "('III1', 'III2', 'VII6')", "('III3', 'III2', 'III7')", "('III3', 'III2', 'VII6')", "('X', 'III3', 'III2')", "('III3', 'III2', 'VII5')", "('III3', 'III2', 'III6')", "('III3', 'III2', 'VII5')"] - [('X', 'III3', 'III2')] vs [('II6', 'V2', 'V1')]
Iter= 3000, Average Loss= 5.295520, Average Accuracy= 2.70%
["('IV6', 'VI1', 'VI2')", "('I6', 'VI1', 'VI2')", "('IV6', 'VI1', 'VI2')", "('VI1', 'VI2', 'I7')", "('III7', 'VI1', 'VI2')", "('VI1', 'VI2', 'I7')", "('X', 'VI1', 'VI2')", "('I6', 'VI1', 'VI2')", "('VI1', 'VI2', 'III6')", "('

Iter= 25000, Average Loss= 3.795834, Average Accuracy= 30.40%
["('X', 'X', 'I5')", "('X', 'VI4', 'X')", "('X', 'X', 'I4')", "('X', 'X', 'II4')", "('X', 'X', 'I4')", "('X', 'X', 'VI3')", "('X', 'I3', 'X')", "('V2', 'V1')", "('VII2', 'V2', 'V1')", "('V2', 'V3', 'V1')"] - [('VII3', 'V2', 'V1')] vs [('IV2', 'IV1', 'I6')]
Iter= 26000, Average Loss= 3.738593, Average Accuracy= 30.30%
["('X', 'V2', 'V1')", "('V2', 'VII6', 'V1')", "('IV6', 'V2', 'V1')", "('II6', 'V2', 'V1')", "('V5', 'V2', 'V1')", "('V2', 'VII5', 'V1')", "('IV5', 'V2', 'V1')", "('II5', 'V2', 'V1')", "('V4', 'V2', 'V1')", "('VII4', 'V2', 'V1')"] - [('V2', 'IV4', 'V1')] vs [('V2', 'X', 'V1')]
Iter= 27000, Average Loss= 3.803202, Average Accuracy= 31.20%
["('I2', 'I3', 'X')", "('I2', 'I3', 'II4')", "('I2', 'I3', 'IV4')", "('I2', 'I3', 'VII3')", "('I2', 'I3', 'X')", "('I2',)", "('I2', 'I3', 'II3')", "('I2', 'I3')", "('I2', 'V3')", "('I2', 'I4')"] - [('I2', 'III4')] vs [('I2', 'X')]
Iter= 28000, Average Loss= 4.047772, Average Accu

Iter= 49000, Average Loss= 2.186606, Average Accuracy= 58.10%
["('III7', 'VI1', 'VI2')", "('VI1', 'VI2', 'I7')", "('X', 'VI1', 'VI2')", "('I6', 'VI1', 'VI2')", "('VI1', 'VI2', 'III6')", "('I6', 'VI1', 'VI2')", "('VI1', 'VI2', 'X')", "('VI1', 'VI2', 'I5')", "('III5', 'VI1', 'VI2')", "('VI1', 'VI2', 'I5')"] - [('X', 'VI1', 'VI2')] vs [('X', 'VI1', 'VI2')]
Iter= 50000, Average Loss= 2.059569, Average Accuracy= 59.10%
["('VII4', 'V2', 'V1')", "('X', 'X', 'II5')", "('X', 'X', 'VII4')", "('X', 'X', 'V5')", "('X', 'X', 'VII5')", "('III1', 'III2', 'II6')", "('III1', 'III2', 'VII5')", "('III1', 'V6', 'III2')", "('III1', 'VI6', 'III2')", "('II2', 'II1', 'II7')"] - [('II2', 'VI6', 'II1')] vs [('II2', 'VI6', 'II1')]
Optimization Finished!
Elapsed time:  1.31881250415 hr
Run on command line.
	tensorboard --logdir=../tmp
Point your web browser to: http://localhost:6006/
10 words: ["('V2', 'II4', 'V1')", "('V2', 'V3', 'V1')", "('VII3', 'V2', 'V1')", "('IV3', 'V2', 'V1')", "('II3', 'V2', 'V1')", "('V2

In [ ]:
tf.saved_model.simple_save(session,'../tmp/model.ckpt',inputs=dictionary,outputs=dictionary)

In [ ]:
saver = tf.train.Saver()
saver.save(session, "/tmp/model.ckpt")